In [5]:
import torch 
import numpy as np
import matplotlib.pyplot as plot
from torch import nn
import torch.nn.functional as F
import sys
import os
cwd = os.getcwd()
sys.path.insert(1, cwd+'/BA_Optimization_ML/MaschineLearning/visualisation')
sys.path.insert(1, cwd+ '/BA_Optimization_ML/MaschineLearning/deeplearningModel')
import LogDataImport
import random
import dataprocessing

from tqdm import tqdm
import time

    
if torch.cuda.is_available() and True:
    device = torch.device("cuda:0")  # you can continue going on here, like cuda:1 cuda:2....etc. 
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")    
    
torch.manual_seed(0)

Running on the CPU


In [6]:
with tqdm(total=4) as pbar:
    #Json
    #Jsondata=json_ImExport.loadData("/Users/leonkiesgen/Documents/Python/Kappa_variation_vmax100.json")
    #Log File (Monteblanco)
    pbar.update(1)
    

    Logdata_MB=LogDataImport.get_data("/Users/leonkiesgen/Documents/Python/mod_local_trajectory/logs/ltpl/2019_12_01/15_00_59_data.csv")

    #Log File (Berlin)
    pbar.update(1)
    ValLogdata_B=LogDataImport.get_data("/Users/leonkiesgen/Documents/Python/mod_local_trajectory/logs/ltpl/2019_12_05/12_19_36_data.csv")

    #Log File (Modena)
    pbar.update(1)
    ValLogdata_M=LogDataImport.get_data("/Users/leonkiesgen/Documents/Python/mod_local_trajectory/logs/ltpl/2019_12_05/15_41_27_data.csv")

    #Log File (zalazone)
    pbar.update(1)
    ValLogdata_Z=LogDataImport.get_data("/Users/leonkiesgen/Documents/Python/mod_local_trajectory/logs/ltpl/2019_12_05/15_51_25_data.csv")


100%|██████████| 4/4 [00:05<00:00,  1.33s/it]


In [59]:
class CNN(nn.Module):

    def __init__(self):
        super(CNN,self).__init__()

        self.c1=nn.Conv1d(in_channels=1,out_channels=10,kernel_size=5,padding=2)
        self.c2=nn.Conv1d(in_channels=10,out_channels=10,kernel_size=5,padding=2)
        self.c3=nn.Conv1d(in_channels=10,out_channels=1,kernel_size=5,padding=2)
    def forward(self,x):
        x=F.relu(self.c1(x))
        x=F.relu(self.c2(x))
        out=torch.sigmoid(self.c3(x))

        return out

In [71]:
inp=torch.rand(1,1,12)
model=CNN()
p=model(inp)
p.squeeze()

tensor([0.4893, 0.4779, 0.4580, 0.4633, 0.4542, 0.4615, 0.4624, 0.4670, 0.4682,
        0.4757, 0.4572, 0.4972], grad_fn=<SqueezeBackward0>)

In [76]:
traindata=dataprocessing.prepareData([ValLogdata_B,ValLogdata_M,ValLogdata_Z],1)

random.shuffle(traindata)
NumSample=len(traindata)
valdata=dataprocessing.prepareData([Logdata_MB])

inputT=traindata[3][0].view(1,1,-1)
print(inputT.shape)

r_out= model(inputT)

print(r_out.squeeze())

70.0
0.001
70.0
0.001
torch.Size([1, 1, 113])
tensor([0.4910, 0.4831, 0.4787, 0.4800, 0.4761, 0.4760, 0.4759, 0.4761, 0.4764,
        0.4768, 0.4774, 0.4780, 0.4787, 0.4792, 0.4797, 0.4802, 0.4808, 0.4813,
        0.4819, 0.4825, 0.4833, 0.4841, 0.4850, 0.4861, 0.4875, 0.4890, 0.4903,
        0.4917, 0.4928, 0.4936, 0.4941, 0.4944, 0.4947, 0.4944, 0.4941, 0.4940,
        0.4942, 0.4942, 0.4940, 0.4938, 0.4937, 0.4939, 0.4942, 0.4945, 0.4945,
        0.4944, 0.4944, 0.4944, 0.4946, 0.4945, 0.4949, 0.4947, 0.4948, 0.4950,
        0.4951, 0.4954, 0.4956, 0.4959, 0.4963, 0.4967, 0.4971, 0.4974, 0.4978,
        0.4982, 0.4985, 0.4987, 0.4992, 0.4996, 0.4999, 0.5003, 0.5006, 0.5010,
        0.5013, 0.5016, 0.5020, 0.5025, 0.5030, 0.5035, 0.5040, 0.5045, 0.5050,
        0.5056, 0.5061, 0.5066, 0.5071, 0.5075, 0.5080, 0.5085, 0.5089, 0.5094,
        0.5099, 0.5103, 0.5109, 0.5114, 0.5121, 0.5122, 0.5131, 0.5136, 0.5140,
        0.5144, 0.5146, 0.5147, 0.5148, 0.5147, 0.5147, 0.5146, 0.5145, 0.

In [78]:
from IPython.core.debugger import set_trace

from torch import optim
criterion = nn.L1Loss()
optimizer = optim.SGD(model.parameters(), lr=0.01)


num_epochs = 10
train_tracker, val_tracker, accuracy_tracker = [], [], []


for i in range(num_epochs):
    cum_loss = 0
    random.shuffle(traindata)
    
    for batch, (inp, vel) in enumerate(traindata,1):

        inp=inp.view(1,1,-1)
        optimizer.zero_grad()
        output = model(inp)
        output=output.squeeze()
        vel=vel.squeeze()
        loss = criterion(output, vel)

        loss.backward()
        optimizer.step()

        cum_loss += loss.item()
        

    train_tracker.append(cum_loss/len(traindata))
    print(f"Epoch({i+1}/{num_epochs}) | Training loss: {cum_loss/len(traindata)} | " ,end="")
    
    test_loss = 0
    num_correct = 0
    total = 0

    random.shuffle(valdata)
    for batch, (inp, vel) in enumerate(valdata,1):
        
        output = model(inp)
        batch_loss = criterion(output, vel)        
        test_loss += batch_loss.item()
    
        
    
    val_tracker.append(test_loss/len(valdata))
    print(f"Val loss: {test_loss/len(valdata)} | ")

/usr/local/lib/python3.7/site-packages/torch/nn/modules/loss.py:88: UserWarning: Using a target size (torch.Size([111])) that is different to the input size (torch.Size([1, 1, 111])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.7/site-packages/torch/nn/modules/loss.py:88: UserWarning: Using a target size (torch.Size([118])) that is different to the input size (torch.Size([1, 1, 118])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.7/site-packages/torch/nn/modules/loss.py:88: UserWarning: Using a target size (torch.Size([124])) that is different to the input size (torch.Size([1, 1, 124])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(

KeyboardInterrupt: 